# PIA PDI V6 EQUIPO 2
## Creación y entrenamiento de la red
### Importar Librerías

In [ ]:
import numpy as np
import os
import tensorflow as tf
import tensorflow.io as tfio
from tensorflow.data import Dataset

### Funciones para obtener los datos
Funciones para obtener la ruta de cada imagen, así como su etiqueta\
Formato de los datos: (ruta, etiqueta)\
Donde la etiqueta es un número entero entre 0 a 5 indicando respectivamente qué letra es, o sí está vacío

| Correspondencia:|  A  |  E  |  I  |  O  |  U  | Nada |
| ---             |:---:|:---:|:---:|:---:|:---:| :---:|
| Etiqueta:       |  0  |  1  |  2  |  3  |  4  |  5   |

Una función para cada banco de imágenes, debido a la naturaleza de las imágenes de cada uno

In [ ]:
#asl-alphabet
def getData(path):
    imagesFolders = ['A', 'E', 'I', 'O', 'U', 'nothing']
    data = []
    for idx, label in enumerate(imagesFolders):
        dataRelativePath = os.path.join(path, label)
        tempDataset = Dataset.list_files(dataRelativePath + '/*.jpg', shuffle=False, seed=1234)
        labels = Dataset.from_tensor_slices([idx]).repeat(len(tempDataset))
        if idx == 0:
            data = Dataset.zip((tempDataset, labels))
        else:
            data = data.concatenate(Dataset.zip((tempDataset, labels)))
    return imagesFolders, data

#synthetic-asl-alphabet
def getData2(path):
    imagesFolders = ['A', 'E', 'I', 'O', 'U', 'Blank']
    data = []
    for idx, label in enumerate(imagesFolders):
        dataRelativePath = os.path.join(path, label)
        tempDataset = Dataset.list_files(dataRelativePath + '/*.png', shuffle=False, seed=1234)
        labels = Dataset.from_tensor_slices([idx]).repeat(len(tempDataset))
        if idx == 0:
            data = Dataset.zip((tempDataset, labels))
        else:
            data = data.concatenate(Dataset.zip((tempDataset, labels)))
    return imagesFolders, data

### Ruta base de cada banco de imágenes

In [ ]:
#asl-alphabet
dataPath = '/kaggle/input/asl-alphabet/' + 'asl_alphabet_train/' * 2

#synthetic-asl-alphabet
dataPath2 = '/kaggle/input/synthetic-asl-alphabet/Train_Alphabet'

### Obtener los conjuntos de imágenes

In [ ]:
#asl-alphabet
labels, dataPaths = getData(dataPath)

#synthetic-asl-alphabet
labels, dataPaths2 = getData2(dataPath2)

### Funciones de preprocesado para cada banco
Datos: (ruta, etiqueta) &rarr; (imagen, etiqueta)

In [ ]:
#asl-alphabet
def preprocess(imagePath, label):
    image = tfio.read_file(imagePath)
    image = tfio.decode_image(image)  
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

#synthetic-asl-alphabet
dataAugmentation = tf.keras.Sequential(
[
    tf.keras.layers.Input(shape=(512, 513, 3)),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1, 0.1),
    tf.keras.layers.CenterCrop(500, 500),
    tf.keras.layers.RandomContrast(0.3),
    tf.keras.layers.Resizing(200, 200)
])

def preprocess2(image, label):
    image = tf.cast(image * 255.0, tf.uint8)
    image = tf.expand_dims(image, 0)
    image = dataAugmentation(image)
    image = tf.squeeze(image, axis=0)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

### Unir los bancos de imágenes

In [ ]:
def datasetMerging(dataset1, dataset2):
    datasetX = dataset1.map(
        preprocess, 
        num_parallel_calls=tf.data.AUTOTUNE)
    
    datasetY = dataset2.map(
        preprocess, 
        num_parallel_calls=tf.data.AUTOTUNE)
    
    datasetY = datasetY.repeat(3)
    datasetY = datasetY.map(
        preprocess2, 
        num_parallel_calls=tf.data.AUTOTUNE)
    
    dataset = datasetX.concatenate(datasetY)
    
    dataset = dataset.cache()
    
    dataset = dataset.shuffle(buffer_size=len(dataset) * 2)
    dataset = dataset.batch(16)
    dataset = dataset.prefetch(8)
    
    return dataset

### Función para separar con el conjunto de imágenes en uno para entrenamiento y otro de validación

In [ ]:
def datasetPartition(dataset):
    trainingBatches = len(dataset) * 7 //10
    testBatches = len(dataset) - trainingBatches
    train = dataset.take(trainingBatches)
    test = dataset.skip(trainingBatches).take(testBatches)
    return train, test

In [ ]:
train, test = datasetPartition(datasetMerging(dataPaths, dataPaths2))

### Importar las funcionalidades necesarias para el modelo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, SeparableConv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Conv2D, Dropout
import tensorflow.keras.optimizers as tfop

### Creación del modelo convolucional

In [ ]:
def createModel(output_nodes):
    model = Sequential(name="Sequential")
    model.add(InputLayer((200, 200, 3)))
    model.add(Conv2D(32, (3,3)))
    model.add(MaxPool2D(2,2))
    
    model.add(SeparableConv2D(32, (3,3)))
    model.add(MaxPool2D(2,2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(SeparableConv2D(64, (3,3)))
    model.add(MaxPool2D(2,2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(SeparableConv2D(128, (3,3)))
    model.add(MaxPool2D(2,2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(SeparableConv2D(256, (3,3)))
    model.add(MaxPool2D(2,2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(tf.keras.layers.GlobalMaxPooling2D())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(output_nodes, activation='sigmoid'))
    
    model.compile(optimizer=tfop.Adam(learning_rate=tfop.schedules.CosineDecay(
                                                    initial_learning_rate=0.01,
                                                    decay_steps=1000,
                                                    alpha=0.1)),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy']
                 )
    return model
model = createModel(len(labels))
model.summary()

### Representación gráfica del modelo
![](https://i.imgur.com/MnEpKUy.png "Representación gráfica del modelo")

### Parámetros para ajustar la taza de aprendizaje o detener el entrenamiento de ser necesario.

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, min_lr=0.00005, verbose=1)
early_s = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1)

### Primer entrenamiento a 10 epochs

In [ ]:
hist = model.fit(train, epochs=10, validation_data=test, callbacks=[reduce_lr, early_s])
model.save('/kaggle/working/pdi10EsigF.h5')

### Segundo entrenamiento a 5 epochs

In [ ]:
hist2 = model.fit(train, epochs=5, validation_data=test, callbacks=[reduce_lr, early_s])
model.save('/kaggle/working/pdi15EsigF.h5')

### Tercer entrenamiento a 5 epochs

In [ ]:
hist3 = model.fit(train, epochs=5, validation_data=test, callbacks=[reduce_lr, early_s])
model.save('/kaggle/working/pdi20EsigF.h5')